# Play Doom Level using Trained PPO Agent  

In [1]:
from utils.env import make_vizdoom_env
from agents.doom_ppo_agent import DoomPpoAgent
import gymnasium as gym

# Creating Environment
num_envs = 1
envs = gym.vector.SyncVectorEnv([ make_vizdoom_env(level='envs/vizdoom/scenarios/basic.cfg', render_mode="human") for i in range(num_envs)])

# Setting up agent
agent = DoomPpoAgent(envs.single_observation_space, 
                     envs.single_action_space,
                     use_gpu=True)
agent.load_models(path='./models/rlhf_pipeline/training_run_2023_08_11_20_13_17/doom_ppo_agent/checkpoint_step_45056')

# Playing config
number_of_episodes = 50

c:\Users\haris\anaconda3\envs\doom-rlhf\lib\site-packages\gymnasium\core.py:297: UserWarning: WARN: env.num_envs to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.num_envs` for environment variables or `env.get_attr('num_envs')` that will search the reminding wrappers.
  logger.warn(


Loading models...
Successfully loaded models!
Updating networks with weights from loaded models...
Successfully updated networks!


In [2]:
import time
import torch
import numpy as np

total_rewards = 0

for episode in range(number_of_episodes):
    observations, infos = envs.reset()
    terminations = np.zeros(num_envs, dtype=np.int32)
    total_episode_rewards = 0

    while not terminations.all():
        # Getting next action and it's value
        action, log_prob, _, value = agent.forward(torch.Tensor(observations))
        observations, rewards, terminations, truncations, infos = envs.step(action)
        total_episode_rewards += rewards

        time.sleep(1/30)

    print(f"Total reward for episode {episode} is {total_episode_rewards}")
    total_rewards += total_episode_rewards

    time.sleep(0.25)

print(f"Average reward per episode is {total_rewards / number_of_episodes}")

e:\Study\University of London\Semester 6\Final Project\doom-rlhf\agents\base_ppo_agent.py:148: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  observations = torch.tensor(observations).to(self.device)


Total reward for episode 0 is [-80.]
Total reward for episode 1 is [58.]
Total reward for episode 2 is [-133.]
Total reward for episode 3 is [-335.]
Total reward for episode 4 is [12.]
Total reward for episode 5 is [-248.]
Total reward for episode 6 is [69.]
Total reward for episode 7 is [68.]
Total reward for episode 8 is [34.]
Total reward for episode 9 is [35.]
Total reward for episode 10 is [-199.]
Total reward for episode 11 is [-86.]
Total reward for episode 12 is [54.]
Total reward for episode 13 is [-74.]
Total reward for episode 14 is [-47.]
Total reward for episode 15 is [31.]
Total reward for episode 16 is [-186.]
Total reward for episode 17 is [-345.]
Total reward for episode 18 is [54.]
Total reward for episode 19 is [-355.]
Total reward for episode 20 is [-236.]
Total reward for episode 21 is [-239.]
Total reward for episode 22 is [-330.]
Total reward for episode 23 is [-355.]
Total reward for episode 24 is [-4.]
Total reward for episode 25 is [-50.]
Total reward for epis

In [3]:
envs.close()